In [2]:
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dash_table.Format import Group
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


from aacCRUD import CRUD


###########################
# Data Manipulation / Model
###########################

usr = 'aacuser'
pw = 'yh0okRBJkaFVCJnw'

str = "mongodb+srv://" + usr + ":" + pw + "@cluster0.cyqsq." + "mongodb.net/?retryWrites=true&w=majority"
#shelter = CRUD(str, 'AAC', 'animals')
shelter = CRUD('mongodb://localhost:27017', 'AAC', 'animals')

df = pd.DataFrame.from_records((shelter.readAll({})))

# Only display the following columns on the table
dft = df[["animal_id", "animal_type", "color", "date_of_birth", "outcome_type", "sex_upon_outcome"]]


#########################
# Dashboard Layout / View
#########################
style = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    
    html.Center(html.B(html.H1('Southern NorthWest Eastern Animal Shelter'))),

    html.Div(id='datatable', children=[]),

    # activated once/week or when page refreshed
    dcc.Interval(id='interval', interval=86400000 * 7, n_intervals=0),

    html.Button("Save Database", id="save-it"),
    html.Button('Add Row', id='adding-rows-btn', n_clicks=0),

    html.Div(id="show-location-map", className='col s12 m6'),
    
    html.Div(id="show-age-graph", children=[]),
    
    html.Div(id="show-outcome-histogram", children=[])
   
])

#Build the table

@app.callback(Output('datatable', 'children'),
              [Input('interval', 'n_intervals')])
def populate_datatable(n_intervals):
    print(n_intervals)
    
    #print(dft.head(20))

    return [
        dash_table.DataTable(
            id='my-table',
            columns=[{'name': x,'id': x,} for x in dft.columns],
            data=df.to_dict('records'),
            editable=True,
            filter_action="native",
            filter_options={"case": "sensitive"},
            sort_action="native",
            sort_mode="multi",
            row_selectable=True,
            row_deletable=False,
            column_selectable=True,
            selected_columns=[],
            selected_rows=[],
            page_current=0,
            page_size=10,
            style_cell={'textAlign': 'left', 'minWidth': '100px',
                        'width': '100px', 'maxWidth': '100px'},
    )
]


#############################################
# Interaction Between Components / Controller
#############################################

#Highlight a row

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Add rows button
@app.callback(
    Output('my-table', 'data'),
    [Input('adding-rows-btn', 'n_clicks')],
    [State('my-table', 'data'),
     State('my-table', 'columns')],
)
def add_row(n_clicks, rows, columns):
    if n_clicks > 0:
        rows.append({c['id']: '' for c in columns})
    return rows


# Save new database button
@app.callback(
    Output("placeholder", "children"),
    Input("save-it", "n_clicks"),
    State("my-table", "data"),
    prevent_initial_call=True
)
def save_data(n_clicks, data):
    dff = pd.DataFrame(data)
    collection.delete_many({})
    collection.insert_many(dff.to_dict('records'))
    return ""


# Create graphs
@app.callback(
    Output('show-outcome-histogram', 'children'),
    Input('my-table', 'data')
)
def build_outcome_histo(data):
    df_graph = pd.DataFrame(data)
    fig_hist1 = px.histogram(df_graph, x='animal_type',color="outcome_type")
    return [
        html.Div(children=[dcc.Graph(figure=fig_hist1)], className="six columns")
        
    ]

@app.callback(
    Output('show-age-graph', 'children'),
    Input('my-table', 'data')
)
def build_age_graph(data):
    
    df_graph = pd.DataFrame(data)
    fig_hist2 = px.histogram(df_graph, x="date_of_birth")
    return [
        html.Div(children=[dcc.Graph(figure=fig_hist2)], className="six columns")
    ]

# Create map of location data
@app.callback(
    Output('show-location-map', "children"),
    Input('my-table', 'data')
)
def build_location_map(data):
    df_map = pd.DataFrame(data)
    fig = px.scatter_mapbox(df_map,lat="location_lat", lon = "location_long",
                        hover_name="animal_id",
                        zoom=11.5,
                        height=700,
                        title='Location',
                        center=dict(lat=30.65, lon=-97.48), 
                        mapbox_style="open-street-map"
                        )
    return [
        html.Div(children=[dcc.Graph(figure=fig)], className = 'col s12 m6')
    ]
    

    

app